# Steps:
    step 1: clean up columns by addressing abbreviations
    Step 2: crate dataframe that contain all public health sentences
    Step 3: for each row, extract sentences that contain at least one keywords that blong to public health keyword lists for each sentences, put a new row into the dataframe, 
    step 4: save the new pandas dataframe into excel file
    Step 5: repeat step 2, 3, and 4 for natural disaster keywords

# Load the data

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_excel("df2010_2020_ph_data.xlsx")

In [3]:
data.shape

(98979, 5)

In [4]:
data.head(3)

,Unnamed: 0,accession#,filing_year,fund_name,principal_risks
0,0,0000004568-10-000004,2010,american balanced fund inc,you may lose money by investing in the fund. t...
1,1,0001193125-10-068753,2010,massmutual premier enhanced index growth fund,the following are the principal risks of the f...
2,2,0001193125-10-068764,2010,massmutual premier discovery value fund,the following are the principal risks of the f...


In [5]:
data['filing_year'].value_counts()

2019    10753
2020    10682
2018    10184
2017     9990
2016     9803
2015     9486
2014     9017
2013     8531
2012     7854
2011     7207
2010     5472
Name: filing_year, dtype: int64

In [6]:
data1 = data.dropna(subset=['principal_risks'])

In [7]:
data1['filing_year'].value_counts()

2020    10592
2019    10451
2018    10184
2017     9990
2016     9803
2015     9486
2014     9017
2013     8531
2012     7854
2011     7207
2010     5472
Name: filing_year, dtype: int64

# Extract Sentences

In [8]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [9]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [10]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    contractions = abbreviations.keys()
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    for sentence_terminator in sentence_terminators:
        t_indices = list(find_all(paragraph, sentence_terminator))
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    for contraction in contractions:
        c_indices = list(find_all(paragraph, contraction))
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    return end


In [11]:
def find_sentences(paragraph):
   replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], ["S. E. C.", "S.E.C."]]
   for items in replacingList:
       paragraph = paragraph.replace(items[0], items[1])
   paragraph = re.sub(' +', ' ', paragraph)
   if paragraph != "":
       if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
           paragraph = paragraph[1:]
   end = True
   sentences = []
   while end > -1:
       end = find_sentence_end(paragraph)
       if end > -1:
           sentences.append(paragraph[end:].strip())
           paragraph = paragraph[:end]
   sentences.append(paragraph)
   sentences.reverse()
   return sentences

#input paragraphs, then split into sentences by using the above function find_sentences()

In [13]:
data1['cleaned_principal_risks']= ""
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    para = data1['principal_risks'].iloc[row]
    sents = find_sentences(para)
    data1['cleaned_principal_risks'].iloc[row] = sents
    
    

<ipython-input-13-1cbc1285c4e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['cleaned_principal_risks']= ""
<ipython-input-13-1cbc1285c4e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['cleaned_principal_risks'].iloc[row] = sents


processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000


In [15]:
data1['cleaned_principal_risks'].head()

0    [you may lose money by investing in the fund.,...
1    [the following are the principal risks of the ...
2    [the following are the principal risks of the ...
3    [it is important to note that this fund seeks ...
4    [the following are the principal risks of the ...
Name: cleaned_principal_risks, dtype: object

In [16]:
data1.to_csv('2020_01_principal_risks_cleaned.csv')

# Public Health Keywords to Search Sentences that Contain one or multiple of these keywords

In [17]:
# Public Health keywords: 
ph_word = ['communicable diseases','health crises','pandemic','respiratory','illness','prevention','epidemic',
           'coronavirus','health crisis','pandemics','sanitation','global health crises',
           'covid','health screenings','pathogens','sars','global health crisis',
           'covid 19','hiv','preparedness','sars cov 2','epidemics','disease','influenza',
           'public health','virus','global health','diseases','mers','quarantines','h1n1','viruses']


In [25]:
import os
import pandas as pd
import glob
import nltk.data
import re
from nltk.tokenize import tokenizer

pubic_health = pd.DataFrame()

for row in range(10): #range(data1):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = tokenizer.tokenize(data1['cleaned_principal_risks'].iloc[row])
    for i in range(len(sentences)):
        for word in ph_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
                                                    'filing_year':data1['filing_year'].iloc[row],
                                                    'fund_name':data1['fund_name'].iloc[row],                                                    
                                                    'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
                   
                break   

ImportError: cannot import name 'tokenizer' from 'nltk.tokenize' (C:\Anaconda3\lib\site-packages\nltk\tokenize\__init__.py)

In [ ]:
pubic_health.shape

In [ ]:
pubic_health.head()

In [ ]:
pubic_health.to_excel("2020_01_public_health_keywords_sentences.xlsx")

# Natural Disaster Keywords to Search Sentences that Contain one or multiple of these keywords

In [ ]:
# Natural Disaster keywords:
nd_word = ['cloud','fire','natural disaster','tornadoes','earthquake',
           'cyclones','flood','natural disasters','tsunami','disaster', 
           'death','floods','seismic','tsunamis','disasters',
           'drought','hurricane','storms','underground',
           'earthquakes','hurricanes','windstorms','volcano'
           'environmental','damage','lightning','droughts','volcanoes']

# For each set of keywords, remove duplicate setences

# Save the datasets into excel files

# Descriptive Statistics of the above three datasets

# Sample code to extract setences

In [ ]:
search_words = ['public health','natural disaster','911']


Extract one sentences before and one sentences after the focus sentence¶ that contains one of the keywords

In [ ]:
import os
import pandas as pd
import glob
import nltk.data
import re
df2 = pd.DataFrame()

for filename in os.listdir(path):
    for f in glob.glob(os.path.join(path,filename, '*.txt')):
        file = open(f,"r")
        data = file.read()
        sentences = tokenizer.tokenize(data)
        for i in range(len(sentences)):
            for word in search_words:
                tokens = sentences[i].lower().split()
                if word in tokens:
                    if i > 0 and i < len(sentences)-1:
                        df2 = df2.append({'id':f,'sentences':re.sub(' +', ' '," ".join(sentences[i-1:i+2]).replace('\n',' '))},ignore_index=True)
                    elif i == 0:
                        df2 = df2.append({'id':f,'sentences':re.sub(' +', ' '," ".join(sentences[0:i+2]).replace('\n',' '))},ignore_index=True)
                    else:
                        df2 = df2.append({'id':f,'sentences':re.sub(' +', ' '," ".join(sentences[i-1:len(sentences)]).replace('\n',' '))},ignore_index=True)
                    break   